In [2]:
%pip install nfl_data_py

Note: you may need to restart the kernel to use updated packages.


In [1]:
import nfl_data_py as nfl
import numpy as np

In [2]:
df = nfl.import_pbp_data(list(range(2000, 2021)))

2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
Downcasting floats.


In [3]:
df['label'] = np.select(
                        [
                            (df['result'] > 0) & (df['posteam'] == df['home_team']),
                            (df['result'] < 0) & (df['posteam'] == df['away_team'])
                        ],
                        [1, 1],
                        default = 0
                       )
df['is_home'] = (df['posteam'] == df['home_team']).astype(int)


/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/4024926356.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['label'] = np.select(
/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/4024926356.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['is_home'] = (df['posteam'] == df['home_team']).astype(int)


In [4]:
df = df[(df['result'] != 0) & (df['qtr'] <= 4)]

# If the starting posteam recieves the second half kick off
df.loc[(df['qtr'] <= 2) & (df['posteam'] == df['defteam'].dropna().iloc[0]), 'recieve_2h_ko'] = 1
df.loc[~((df['qtr'] <= 2) & (df['posteam'] == df['defteam'].dropna().iloc[0])), 'recieve_2h_ko'] = 0

# Convert to the posteam for spread
df['posteam_spread'] = np.where(df['is_home'] == 1, df['spread_line'], -1 * df['spread_line'])
df['elapsed_share'] = (3600 - df['game_seconds_remaining']) / 3600

# Exponential decay of the spread. It weighs more towards the start
df['spread_time'] = df['posteam_spread'] * np.exp(-4*df['elapsed_share']) # Weights the posteam spread over time



/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/2883950221.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[(df['qtr'] <= 2) & (df['posteam'] == df['defteam'].dropna().iloc[0]), 'recieve_2h_ko'] = 1
/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/2883950221.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['posteam_spread'] = np.where(df['is_home'] == 1, df['spread_line'], -1 * df['spread_line'])
/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/2883950221.py:9: Per

In [16]:
model_data = df[[
    # Explanatory variables
    'label', 'game_id', 'season',
    
    
    # Features
    'recieve_2h_ko', 'spread_time', 'is_home', 'half_seconds_remaining', 'game_seconds_remaining', 
    'score_differential','down', 'ydstogo', 'yardline_100', 
    'posteam_timeouts_remaining', 'defteam_timeouts_remaining'
    
    
]]

In [22]:
model_data.dropna(inplace=True)
model_data.reset_index(drop=True, inplace=True)

/var/folders/0m/p03jcv9j4td3thddmd99m0300000gp/T/ipykernel_59191/70319802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.dropna(inplace=True)


In [25]:
import pandas as pd
model_data = pd.read_csv('nfl-pred-data.csv')